# __Proyecto de clustering de segmentación de países por datos socio-económicos__

## Importar librerías

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as  sns

## Carga de datos

In [7]:
df_country = pd.read_csv('Country-data.csv')
df_country.head()

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,Afghanistan,90.2,10.0,7.58,44.9,1610,9.44,56.2,5.82,553
1,Albania,16.6,28.0,6.55,48.6,9930,4.49,76.3,1.65,4090
2,Algeria,27.3,38.4,4.17,31.4,12900,16.10,76.5,2.89,4460
3,Angola,119.0,62.3,2.85,42.9,5900,22.40,60.1,6.16,3530
4,Antigua and Barbuda,10.3,45.5,6.03,58.9,19100,1.44,76.8,2.13,12200


In [8]:
df_country.shape

(167, 10)

> El dataset esta constituido de 10 columnas y 167 filas

## Conociendo los datos

La organizacion HELP International es una ONG humanitaria internacional comprometida a combatir la pobreza y proporcionar a la población de los países atrasados ​​servicios básicos y socorro en tiempos de desastres y calamidades naturales.

HELP International ha logrado recaudar alrededor de 10 millones de dólares. Ahora, el director ejecutivo de la ONG debe decidir cómo utilizar este dinero de forma estratégica y eficaz. Por lo tanto, debe elegir los países que más necesitan ayuda. Por lo tanto, su trabajo como científico de datos consiste en categorizar los países utilizando factores socioeconómicos y sanitarios que determinan su desarrollo general. A continuación, debe sugerir los países en los que el director ejecutivo debe centrarse más.

El dataset con el que se cuenta tiene la siguiente informacion:

- _country_: pais.

- _child_mort_: Muerte de niños menores de 5 años por cada 1000 nacidos vivos.

- _exports_: Exportaciones de bienes y servicios per cápita. Expresadas como porcentaje del PIB per cápita.

- _health_: Gasto total en salud per cápita. Expresado como porcentaje del PIB per cápita.

- _imports_: Importaciones de bienes y servicios per cápita. Expresadas como porcentaje del PIB per cápita.

- _income_: Ingreso neto por persona.

- _inflation_: La medición de la tasa de crecimiento anual del PIB total.

- _life_expec_: El número promedio de años que viviría un recién nacido si los patrones de mortalidad actuales se mantuvieran

- _total_fer_: El número de hijos que tendría cada mujer si las tasas actuales de fertilidad por edad se mantuvieran iguales.

- _gdpp_: El PIB per cápita. Se calcula dividiendo el PIB total entre la población total.

## Manejando valores nulos

## Validando duplicados

## Análisis de distribución

## Escalar datos

## PCA

## Importar librerías para clustering

## K-means

### PCA

### Sin PCA

## Agglomerativen hierarchical clustering

### PCA

### Sin PCA

## DBSCAN

### PCA

### Sin PCA

## Evaluación de resultados